# Новые возможности Python 3.7 и 3.8

## Классы данных (3.7)

https://habr.com/ru/post/415829/

Часто в конструкторе класса можно наблюдать следующую ситуацию:

In [ ]:
class SomeThing:
    def __init__(self, value1, value2):
        self.value1 = value1
        self.value2 = value2

В Python 3.7 появились классы данных, в которых не нужно писать все эти однотипные присвоения. Они задаются декотратором `@dataclass`

In [ ]:
from dataclasses import dataclass

@dataclass
class SomeThing:
    value1: int
    value2: str
        

s = SomeThing(1, "abc")
s.value1, s.value2

Аннотации типов обязательны, иначе поля игнорируются декоратором (да и NameError возникает)

In [ ]:
@dataclass
class SomeThing:
    value1: int
    value2: str
    value3
        

s = SomeThing(1, "abc")
s.value1, s.value2, s.value3

### make_dataclass

Библиотека dataclass предоставляет функцию, которая позволяет создавать класс данных следующим образом:

In [ ]:
from dataclasses import make_dataclass

SomeThing = make_dataclass("SomeThing", ["value1", "value2"])
s = SomeThing(1, "abc")
s.value1, s.value2

### Значения по умолчанию

В синтаксисе датакласса можно указать значения по умолчанию:

In [ ]:
@dataclass
class SomeThing:
    value1: int = 1
    value2: str = "abc"
        
a = SomeThing(45)
b = SomeThing(value2="abcde")

print(a.value1, a.value2)
print(b.value1, b.value2)

Но надо помнить о том, что аргументы по умолчанию всегда должны следовать после всех позиционных аргументов. Это касается и конструктора, генерируемого декоратором dataclass.

In [ ]:
@dataclass
class SomeThing:
    value1: int = 1
    value2: str
        
a = SomeThing(45)
b = SomeThing(value2="abcde")

print(a.value1, a.value2)
print(b.value1, b.value2)

### Frozen Data Class

Отличный способ, чтобы хранить константы: объекты датаклассов, созданных таким образом, неизменяемы.

In [ ]:
@dataclass(frozen=True)
class SomeThing:
    value1: int
    value2: str
        
        
s = SomeThing(value1=256, value2="abc")
s.value2 = "abcde"

In [ ]:
s.value3 = 2

### Параметры класса данных

- `init: bool = True` - создать или не создать конструктор
- `repr: bool = True` - создать или не создать `__repr__`
- `eq: bool = True` - создать или не создать метод `__eq__`
- `order: bool = False` - создать или не создать методы сравнения объектов
- `unsafe_hash: bool: False` - создать или не создать метод `__hash__`. Само создание метода зависит также от параметров `eq` и `frozen`
- `frozen: bool = False` - запрет изменения атрибутов класса

Посмотрим, что из себя представляют `__eq__`, `__lt__` и остальные, сгенерированные автоматически

In [ ]:
@dataclass(eq=True, order=True, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str

SomeThing(1, 2) == SomeThing(1, 2)

In [ ]:
SomeThing(1, 2) > SomeThing(2, 1)

In [ ]:
SomeThing(1, 2) < SomeThing(2, 1)

По умолчанию у таких классов сравниваются кортежи значений в том порядке, в котором они были изначально заданы. Посмотреть OrderedDict параметров можно через поле `__dataclass_fields__`

In [ ]:
SomeThing(100, "abc").__dataclass_fields__

А что с хешами?

In [ ]:
@dataclass(eq=True, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

In [ ]:
@dataclass(eq=False, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

In [ ]:
@dataclass(frozen=True, eq=True, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

In [ ]:
@dataclass(frozen=True, eq=False, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

In [ ]:
@dataclass(frozen=True, eq=False)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

In [ ]:
@dataclass(frozen=False, eq=False, unsafe_hash=False)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

In [ ]:
@dataclass(frozen=False, eq=True, unsafe_hash=False)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

In [ ]:
@dataclass(frozen=True, eq=True, unsafe_hash=False)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

### Изменяемые значения по умолчанию

Как мы знаем, использовать изменяемые объекты в качестве значений по умолчанию - плохая идея, поскольку они инстанцируются только один раз при объявлении функции. Dataclass это учитывает:

In [ ]:
from typing import List

@dataclass
class SomeThing:
    value: List[int] = []

Библиотека предлагает использовать default_factory:

In [ ]:
from dataclasses import field

@dataclass
class SomeThing:
    value: List[int] = field(default_factory=list)
        
SomeThing().value

#### Параметры field
- `default`: значение по умолчанию. Этот параметр необходим, так как вызов `field` заменяет задание значения поля по умолчанию
- `init`: включает (задан по умолчанию) использование поля в методе `__init__`
- `repr`: включает (задан по умолчанию) использование поля в методе `__repr__`
compare включает (задан по умолчанию) использование поля в методах сравнения (`__eq__`, `__le__` и других)
- `hash`: может быть булевое значение или None. Если он равен True, поле используется при вычислении хэша. Если указано None (по умолчанию) — используется значение параметра compare.
Одной из причин указать `hash=False` при заданном `compare=True` может быть сложность вычисления хэша поля при том, что оно необходимо для сравнения.
- `metadata`: произвольный словарь или None. Значение оборачивается в MappingProxyType, чтобы оно стало неизменяемым. Этот параметр не используется самими классами данных и предназначен для работы сторонних расширений.


### Обработка после инициализации

В классах данных автоматически создается метод `__init__`, в котором исплоняется код присвоения значений в поля объекта: `self.value = value`. Но что если мы хотим использовать датакласс, но дополнить конструктор какими-то еще действиями? Для этого можем задать метод `__post_init__`

In [ ]:
@dataclass
class Book:
    title: str
    author: str
    desc: str = None

    def __post_init__(self):
        self.desc = self.desc or "`%s` by %s" % (self.title, self.author)
        
        
Book("Название", "Автор").desc

В этом методе можно использовать дополнительные аргументы конструктора, которые не нужно записывать в self. Для этого предназначен класс `dataclasses.InitVar`:

In [ ]:
from dataclasses import InitVar

@dataclass
class Book:
    title: str
    author: str
    gen_desc: InitVar[bool] = True
    desc: str = None

    def __post_init__(self, gen_desc: str):
        if gen_desc and self.desc is None:
            self.desc = "`%s` by %s" % (self.title, self.author)
            
            
print(Book("Название", "Автор", True).desc)
print(Book("Название", "Автор", False).desc)

### Наследование в датаклассах

Если датакласс наследуется от другого датакласса, то он складывает OrderedDict'ы обоих классов и использует полученный общий OrderedDict во всех генерируемых методах.

In [ ]:
from typing import Any


@dataclass
class BaseBook:
    title: Any = None
    author: str = None

@dataclass
class Book(BaseBook):
    desc: str = None
    title: str = "Unknown"
        
        
Book()

In [ ]:
Book().__dataclass_fields__

# Новое в Python 3.8

https://habr.com/ru/post/483276/

## Моржовый оператор

Оператор присваивания - как в паскале. Служит для улучшения читаемости кода.

In [ ]:
a = 6

# Код ниже присваивает b значение a ** 2
# и проверяет, если b > 0
if (b := a ** 2) > 0: 
    print(f'Квадрат {a} это {b}.') # Квадрат 6 это 36.

## Только позиционные аргументы

Всё, что записано в сигнатуре функции до знака /, можно передавать только как позиционные аргументы. По имени - нельзя, будет ошибка.

In [ ]:
def my_func(a, b, /, c, d, *, e, f):
    return a+b+c+d+e+f
  
my_func(1, 2, 3, 4, 5, 6)         # ошибка: e, f должны быть именованными
my_func(a=1, b=2, 3, 4, e=5, f=6) # ошибка: a, b должны быть позиционными
my_func(1, 2, c=3, 4, e=5, f=6)   # returns 21
my_unc(1, 2, c=3, d=4, e=5, f=6)  # returns 21

## Улучшенный дебаг print'ом =)

Новый синтаксис f-строк позволяет выводить сразу имя переменной и ее значение:

In [ ]:
pi = 3  # В военное время может быть и так

print(f'pi={pi}')  # так мы делали раньше
print(f'{pi=}')     # а так можно делать теперь

## reversed()

теперь можно вызывать не только над объектами, в которых реализован метод `__reversed__`, но и над теми, в которых описаны `__len__` и `__getitem__`. В числе таких объектов - словари.

## Получение метаданных из других модулей

Новый модуль importlib.metadata позволит получать метаданные (например, версию) из сторонних пакетов.

## Использование continue в блоке finally

In [ ]:
for i in range(2):
    try:
        print(i)
    finally:
        print('Тест.')
        continue
        print('Эту строку вы не увидите.')